In [ ]:
import pandas as pd

# 範例：讀入你的檔案
df = pd.read_csv('北大學生感情狀況-正式版 (回覆) - 表單回應 1.csv')

# 用 bfill（向後填值）＋取第一個欄位，得到每列的第一個非空值
cols = [
    '您目前就讀的科系為？',
    '您目前就讀的科系為？.1',
    '您目前就讀的科系為？.2',
    '您目前就讀的科系為？.3',
    '您目前就讀的科系為？.4',
]
df['合併科系'] = df[cols].bfill(axis=1).iloc[:, 0]

# 刪掉原來的四個欄位
df = df.drop(columns=cols)

# (可選) 存回 CSV
df.to_csv('survey_data_merged.csv', index=False)

# 看一下結果
print(df[['合併科系']].head())
df = pd.read_csv('北大學生感情狀況-正式版 (回覆) - 表單回應 1.csv')

for i in range(1, 5):
    string = f'您目前就讀的科系為？.{i}'
    print(string)
    df['您目前就讀的科系為？'] = df['您目前就讀的科系為？'].fillna(df[string])
    

df['您目前就讀的科系為？'].head(300)
for i in range(1, 5):
    string = f'您目前就讀的科系為？.{i}'
    df = df.drop(columns=[string])

df.to_csv('test.csv',index=False)

import pandas as pd
import numpy as np

df = pd.read_csv('test.csv')

for i in range(len(df)):
    if pd.isna(df.loc[i, '您目前就讀的科系為？']):
        print(df.loc[i, '您目前就讀的院所為？'])

df['您目前就讀的科系為？'] = df['您目前就讀的科系為？'].fillna('法律系')

for i in range(len(df)):
    if pd.isna(df.loc[i, '您目前就讀的科系為？']):
        print(df.loc[i, '您目前就讀的院所為？'])
df['截至目前為止，您曾經有過幾段戀愛關係？（含現在）'] = df['截至目前為止，您曾經有過幾段戀愛關係？（含現在）'].replace('3或以上', '3')
df.to_csv('整理後.csv')

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# 檢查檔案是否存在
if not os.path.exists('整理後.csv'):
    raise FileNotFoundError("找不到 '整理後.csv' 檔案，請確認檔案路徑！")

# 1. 讀入資料
df = pd.read_csv('整理後.csv')

# 2. 準備欄位清單（排除時間戳記）
choices = df.select_dtypes(include=['object', 'category']).columns.tolist()
choices = [c for c in choices if c != '時間戳記']
if not choices:
    raise ValueError("資料中沒有分類變數（object 或 category 型態）！")

# 3. 建立 widgets
n_var = widgets.Dropdown(options=[1, 2, 3], value=2, description='變數數：')
x_dd = widgets.Dropdown(options=choices, description='Var 1：')
y_dd = widgets.Dropdown(options=choices, description='Var 2：')
z_dd = widgets.Dropdown(options=choices, description='Var 3：')
out = widgets.Output()

# 初始顯示狀態
y_dd.layout.display = 'flex'
z_dd.layout.display = 'none'

def update_y_dd(change):
    y_dd.options = [c for c in choices if c != x_dd.value and c != z_dd.value]
def update_z_dd(change):
    z_dd.options = [c for c in choices if c != x_dd.value and c != y_dd.value]

x_dd.observe(update_y_dd, names='value')
y_dd.observe(update_z_dd, names='value')

def render(change):
    # 動態顯示/隱藏下拉
    y_dd.layout.display = 'flex' if n_var.value >= 2 else 'none'
    z_dd.layout.display = 'flex' if n_var.value == 3 else 'none'
    
    with out:
        clear_output(wait=True)
        # 一變數：只畫 Var1 分布的圓餅
        if n_var.value == 1:
            counts = df[x_dd.value].value_counts()
            print(f"── 單變數分布：{x_dd.value} ──")
            display(counts.to_frame('Count'))
            plt.figure(figsize=(15, 15))
            counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
            plt.title(f'{x_dd.value} 分布')
            plt.ylabel('')
            plt.tight_layout()
            plt.show()
        
        # 兩變數：卡方 + 長條圖 + 每個 Var1 類別的圓餅
        elif n_var.value == 2:
            print(f"── {x_dd.value} vs {y_dd.value} 卡方檢定 ──")
            ct = pd.crosstab(df[x_dd.value], df[y_dd.value])
            if ct.size == 0 or ct.shape[0] < 1 or ct.shape[1] < 1:
                print("交叉表無效，無法執行卡方檢定！")
                return
            chi2, p, _, expected = chi2_contingency(ct)
            if (expected < 5).sum() > 0.2 * expected.size:
                print("警告：預期頻次過低，卡方檢定結果可能不可靠！")
            print(f"χ²={chi2:.2f}, p={p:.4f} → {'顯著' if p<0.05 else '不顯著'}")
            display(ct)
            # 長條圖
            plt.figure(figsize=(15, 15))
            ct.plot(kind='bar', stacked=False)
            plt.title(f'{x_dd.value} vs {y_dd.value}')
            plt.xticks(rotation=0)
            plt.tight_layout()
            plt.show()
            # 圓餅圖
            for x_val in ct.index:
                plt.figure(figsize=(10,10))
                ct.loc[x_val].plot(
                    kind='pie',
                    autopct='%1.1f%%',
                    startangle=90,
                    title=f'{x_dd.value}={x_val}'
                )
                plt.ylabel('')
                plt.tight_layout()
                plt.show()
        
        # 三變數：整體 + 分組卡方，長條 & 圓餅
        else:
            print(f"── 整體 {x_dd.value} vs {y_dd.value} 卡方檢定 ──")
            overall = pd.crosstab(df[x_dd.value], df[y_dd.value])
            if overall.size == 0 or overall.shape[0] < 1 or overall.shape[1] < 1:
                print("交叉表無效，無法執行卡方檢定！")
                return
            chi2, p, _, expected = chi2_contingency(overall)
            if (expected < 5).sum() > 0.2 * expected.size:
                print("警告：預期頻次過低，卡方檢定結果可能不可靠！")
            print(f"χ²={chi2:.2f}, p={p:.4f} → {'顯著' if p<0.05 else '不顯著'}")
            display(overall)
            # 整體圓餅
            plt.figure(figsize=(6, 6))
            df[x_dd.value].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
            plt.title(f'{x_dd.value} 單變數分布')
            plt.ylabel('')
            plt.tight_layout()
            plt.show()
            
            # 分組
            for z_val, sub in df.groupby(z_dd.value):
                print(f"\n── {z_dd.value}={z_val} 卡方檢定 ──")
                ct = pd.crosstab(sub[x_dd.value], sub[y_dd.value])
                if ct.size == 0 or ct.shape[0] < 1 or ct.shape[1] < 1:
                    print("交叉表無效，無法執行卡方檢定！")
                    continue
                chi2, p, _, expected = chi2_contingency(ct)
                if (expected < 5).sum() > 0.2 * expected.size:
                    print("警告：預期頻次過低，卡方檢定結果可能不可靠！")
                print(f"χ²={chi2:.2f}, p={p:.4f} → {'顯著' if p<0.05 else '不顯著'}")
                display(ct)
                # 長條圖
                plt.figure(figsize=(15, 15))
                ct.plot(kind='bar', stacked=False)
                plt.title(f'{z_dd.value}={z_val}')
                plt.xticks(rotation=0)
                plt.tight_layout()
                plt.show()
                # 圓餅圖
                for x_val in ct.index:
                    plt.figure(figsize=(10,10))
                    ct.loc[x_val].plot(
                        kind='pie',
                        autopct='%1.1f%%',
                        startangle=90,
                        title=f'{z_dd.value}={z_val}, {x_dd.value}={x_val}'
                    )
                    plt.ylabel('')
                    plt.tight_layout()
                    plt.show()

# 綁定事件
for w in (n_var, x_dd, y_dd, z_dd):
    w.observe(render, names='value')

# 顯示
display(widgets.VBox([
    n_var,
    widgets.HBox([x_dd, y_dd, z_dd]),
    out
]))